In [1]:
import pandas as pd
from sklearn.ensemble import HistGradientBoostingClassifier
import numpy as np
from sklearn import tree
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression #, Lasso, Ridge, LassoCV,BayesianRidge
import statsmodels.formula.api as sm
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
import matplotlib.pylab as plt
from sklearn.neural_network import MLPClassifier
from dmba import classificationSummary
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.neighbors import KNeighborsClassifier

In [2]:
space_df=pd.read_csv('C:/Users/arun2/Downloads/spaceship-titanic/train.csv',usecols = range(1,14))

In [3]:
space_df

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False
8689,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False
8690,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True
8691,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False


In [4]:
space_df = space_df.drop(['Name'], axis=1)

In [5]:
space_df.dtypes

HomePlanet       object
CryoSleep        object
Cabin            object
Destination      object
Age             float64
VIP              object
RoomService     float64
FoodCourt       float64
ShoppingMall    float64
Spa             float64
VRDeck          float64
Transported        bool
dtype: object

In [6]:
space_df.isnull().sum()

HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Transported       0
dtype: int64

In [7]:
objList = space_df.select_dtypes(include = "object").columns
print (objList)

Index(['HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'VIP'], dtype='object')


In [8]:
space_df = pd.get_dummies(space_df, columns = ['HomePlanet', 'Destination'])
print(space_df)

     CryoSleep     Cabin   Age    VIP  RoomService  FoodCourt  ShoppingMall  \
0        False     B/0/P  39.0  False          0.0        0.0           0.0   
1        False     F/0/S  24.0  False        109.0        9.0          25.0   
2        False     A/0/S  58.0   True         43.0     3576.0           0.0   
3        False     A/0/S  33.0  False          0.0     1283.0         371.0   
4        False     F/1/S  16.0  False        303.0       70.0         151.0   
...        ...       ...   ...    ...          ...        ...           ...   
8688     False    A/98/P  41.0   True          0.0     6819.0           0.0   
8689      True  G/1499/S  18.0  False          0.0        0.0           0.0   
8690     False  G/1500/S  26.0  False          0.0        0.0        1872.0   
8691     False   E/608/S  32.0  False          0.0     1049.0           0.0   
8692     False   E/608/S  44.0  False        126.0     4688.0           0.0   

         Spa  VRDeck  Transported  HomePlanet_Earth

In [9]:
space_df = space_df.apply(lambda x: x.fillna(x.value_counts().index[0]))
space_df

,CryoSleep,Cabin,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
0,False,B/0/P,39.0,False,0.0,0.0,0.0,0.0,0.0,False,0,1,0,0,0,1
1,False,F/0/S,24.0,False,109.0,9.0,25.0,549.0,44.0,True,1,0,0,0,0,1
2,False,A/0/S,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False,0,1,0,0,0,1
3,False,A/0/S,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False,0,1,0,0,0,1
4,False,F/1/S,16.0,False,303.0,70.0,151.0,565.0,2.0,True,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,False,A/98/P,41.0,True,0.0,6819.0,0.0,1643.0,74.0,False,0,1,0,1,0,0
8689,True,G/1499/S,18.0,False,0.0,0.0,0.0,0.0,0.0,False,1,0,0,0,1,0
8690,False,G/1500/S,26.0,False,0.0,0.0,1872.0,1.0,0.0,True,1,0,0,0,0,1
8691,False,E/608/S,32.0,False,0.0,1049.0,0.0,353.0,3235.0,False,0,1,0,1,0,0


In [10]:
space_df.isnull().sum()

CryoSleep                    0
Cabin                        0
Age                          0
VIP                          0
RoomService                  0
FoodCourt                    0
ShoppingMall                 0
Spa                          0
VRDeck                       0
Transported                  0
HomePlanet_Earth             0
HomePlanet_Europa            0
HomePlanet_Mars              0
Destination_55 Cancri e      0
Destination_PSO J318.5-22    0
Destination_TRAPPIST-1e      0
dtype: int64

In [12]:
objList = space_df.select_dtypes(include = "object").columns
print (objList)

Index(['Cabin'], dtype='object')


In [13]:
#Label Encoding for object to numeric conversion
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

for values in objList:
    space_df[values] = le.fit_transform(space_df[values].astype(str))

space_df = space_df

In [14]:
space_df

,CryoSleep,Cabin,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
0,False,149,39.0,False,0.0,0.0,0.0,0.0,0.0,False,0,1,0,0,0,1
1,False,2184,24.0,False,109.0,9.0,25.0,549.0,44.0,True,1,0,0,0,0,1
2,False,1,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False,0,1,0,0,0,1
3,False,1,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False,0,1,0,0,0,1
4,False,2186,16.0,False,303.0,70.0,151.0,565.0,2.0,True,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,False,146,41.0,True,0.0,6819.0,0.0,1643.0,74.0,False,0,1,0,1,0,0
8689,True,5280,18.0,False,0.0,0.0,0.0,0.0,0.0,False,1,0,0,0,1,0
8690,False,5285,26.0,False,0.0,0.0,1872.0,1.0,0.0,True,1,0,0,0,0,1
8691,False,2131,32.0,False,0.0,1049.0,0.0,353.0,3235.0,False,0,1,0,1,0,0


In [22]:
predictors = space_df.loc[:, space_df.columns != 'Transported']

In [23]:
X = predictors

In [24]:
predictors

,CryoSleep,Cabin,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
0,False,149,39.0,False,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0,1
1,False,2184,24.0,False,109.0,9.0,25.0,549.0,44.0,1,0,0,0,0,1
2,False,1,58.0,True,43.0,3576.0,0.0,6715.0,49.0,0,1,0,0,0,1
3,False,1,33.0,False,0.0,1283.0,371.0,3329.0,193.0,0,1,0,0,0,1
4,False,2186,16.0,False,303.0,70.0,151.0,565.0,2.0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,False,146,41.0,True,0.0,6819.0,0.0,1643.0,74.0,0,1,0,1,0,0
8689,True,5280,18.0,False,0.0,0.0,0.0,0.0,0.0,1,0,0,0,1,0
8690,False,5285,26.0,False,0.0,0.0,1872.0,1.0,0.0,1,0,0,0,0,1
8691,False,2131,32.0,False,0.0,1049.0,0.0,353.0,3235.0,0,1,0,1,0,0


In [25]:
outcome = space_df.loc[:, space_df.columns == 'Transported']

In [26]:
y = outcome

In [27]:
outcome

,Transported
0,False
1,True
2,False
3,False
4,True
...,...
8688,False
8689,False
8690,True
8691,False


In [28]:
train_X, valid_X, train_y, valid_y = train_test_split(X, y, test_size=0.3,random_state=1) 

In [29]:
scaler = StandardScaler()
train_X = scaler.fit_transform(train_X)
valid_X = scaler.transform(valid_X)

In [30]:
data={'Data Set':['train_X', 'valid_X','train_y','valid_y'], 'Shape': [train_X.shape, valid_X.shape, train_y.shape, valid_y.shape]}
df=pd.DataFrame(data)
df

,Data Set,Shape
0,train_X,"(6085, 15)"
1,valid_X,"(2608, 15)"
2,train_y,"(6085, 1)"
3,valid_y,"(2608, 1)"


In [31]:
gbc = GradientBoostingClassifier(n_estimators=10000, learning_rate=1.0,
      max_depth=1, random_state=42).fit(train_X,train_y)
gbc.score(train_X,train_y)

D:\Anaconda py\lib\site-packages\sklearn\ensemble\_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.8632703368940017

In [32]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()
model.fit(train_X,train_y)
model.score(train_X,train_y)

0.999671322925226

In [33]:
space_df_test =pd.read_csv('C:/Users/arun2/Downloads/spaceship-titanic/test.csv',usecols = range(1,13))
space_df_test.head()

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning
1,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers
2,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus
3,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter
4,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez


In [34]:
space_df_test.isnull().sum()

HomePlanet       87
CryoSleep        93
Cabin           100
Destination      92
Age              91
VIP              93
RoomService      82
FoodCourt       106
ShoppingMall     98
Spa             101
VRDeck           80
Name             94
dtype: int64

In [35]:
objList = space_df_test.select_dtypes(include = "object").columns
print (objList)

Index(['HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'VIP', 'Name'], dtype='object')


In [36]:
space_df_test = pd.get_dummies(space_df_test, columns = ['HomePlanet', 'Destination'])
print(space_df_test)

     CryoSleep     Cabin   Age    VIP  RoomService  FoodCourt  ShoppingMall  \
0         True     G/3/S  27.0  False          0.0        0.0           0.0   
1        False     F/4/S  19.0  False          0.0        9.0           0.0   
2         True     C/0/S  31.0  False          0.0        0.0           0.0   
3        False     C/1/S  38.0  False          0.0     6652.0           0.0   
4        False     F/5/S  20.0  False         10.0        0.0         635.0   
...        ...       ...   ...    ...          ...        ...           ...   
4272      True  G/1496/S  34.0  False          0.0        0.0           0.0   
4273     False       NaN  42.0  False          0.0      847.0          17.0   
4274      True   D/296/P   NaN  False          0.0        0.0           0.0   
4275     False   D/297/P   NaN  False          0.0     2680.0           0.0   
4276      True  G/1498/S  43.0  False          0.0        0.0           0.0   

         Spa  VRDeck              Name  HomePlanet_

In [37]:
space_df_test = space_df_test.apply(lambda x: x.fillna(x.value_counts().index[0]))
space_df_test

,CryoSleep,Cabin,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
0,True,G/3/S,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning,1,0,0,0,0,1
1,False,F/4/S,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers,1,0,0,0,0,1
2,True,C/0/S,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus,0,1,0,1,0,0
3,False,C/1/S,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter,0,1,0,0,0,1
4,False,F/5/S,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,True,G/1496/S,34.0,False,0.0,0.0,0.0,0.0,0.0,Jeron Peter,1,0,0,0,0,1
4273,False,G/160/P,42.0,False,0.0,847.0,17.0,10.0,144.0,Matty Scheron,1,0,0,0,0,1
4274,True,D/296/P,18.0,False,0.0,0.0,0.0,0.0,0.0,Jayrin Pore,0,0,1,1,0,0
4275,False,D/297/P,18.0,False,0.0,2680.0,0.0,0.0,523.0,Kitakan Conale,0,1,0,0,0,0


In [41]:
objList = space_df_test.select_dtypes(include = "object").columns
print (objList)

Index(['Cabin'], dtype='object')


In [40]:
space_df_test = space_df_test.drop(['Name'], axis=1)

In [42]:
#Label Encoding for object to numeric conversion
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

for values in objList:
    space_df_test[values] = le.fit_transform(space_df_test[values].astype(str))

space_df_test = space_df_test

In [43]:
space_df_test

,CryoSleep,Cabin,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
0,True,2784,27.0,False,0.0,0.0,0.0,0.0,0.0,1,0,0,0,0,1
1,False,1867,19.0,False,0.0,9.0,0.0,2823.0,0.0,1,0,0,0,0,1
2,True,257,31.0,False,0.0,0.0,0.0,0.0,0.0,0,1,0,1,0,0
3,False,259,38.0,False,0.0,6652.0,0.0,181.0,585.0,0,1,0,0,0,1
4,False,1940,20.0,False,10.0,0.0,635.0,0.0,0.0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,True,2679,34.0,False,0.0,0.0,0.0,0.0,0.0,1,0,0,0,0,1
4273,False,2691,42.0,False,0.0,847.0,17.0,10.0,144.0,1,0,0,0,0,1
4274,True,602,18.0,False,0.0,0.0,0.0,0.0,0.0,0,0,1,1,0,0
4275,False,603,18.0,False,0.0,2680.0,0.0,0.0,523.0,0,1,0,0,0,0


In [44]:
y_test = gbc.predict(space_df_test)

D:\Anaconda py\lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but GradientBoostingClassifier was fitted without feature names
  warnings.warn(


In [45]:
y_test

array([False, False, False, ..., False, False, False])

In [46]:
y_test_dtc = model.predict(space_df_test)

D:\Anaconda py\lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(


In [47]:
y_test_dtc

array([ True, False,  True, ...,  True, False,  True])

In [48]:
space_df_final = pd.read_csv('C:/Users/arun2/Downloads/spaceship-titanic/test.csv')

In [49]:
space_df_final

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,9266_02,Earth,True,G/1496/S,TRAPPIST-1e,34.0,False,0.0,0.0,0.0,0.0,0.0,Jeron Peter
4273,9269_01,Earth,False,NaN,TRAPPIST-1e,42.0,False,0.0,847.0,17.0,10.0,144.0,Matty Scheron
4274,9271_01,Mars,True,D/296/P,55 Cancri e,NaN,False,0.0,0.0,0.0,0.0,0.0,Jayrin Pore
4275,9273_01,Europa,False,D/297/P,NaN,NaN,False,0.0,2680.0,0.0,0.0,523.0,Kitakan Conale


In [50]:
prediction_value = pd.DataFrame(y_test_dtc,columns=['Transported'])
#prediction_probability_value = pd.DataFrame(y_pred_prob,columns=['predicted_probability'])
var =pd.DataFrame(space_df_final[['PassengerId']])
dataset_test_col = pd.concat([var,prediction_value], axis=1)
dataset_test_col.to_csv("submissions_space.csv",index=False)